In [1]:
# !pip install keras_cv_attention_models

In [2]:
import evals
from tensorflow import keras
import tensorflow as tf
import IJB_evals
import matplotlib.pyplot as plt
import keras_cv_attention_models
import GhostFaceNets, GhostFaceNets_with_Bias

[WARNING] Setting TF_USE_LEGACY_KERAS=1. Make sure this is ahead of importing tensorflow or keras.


In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[]


2 ways to load the model

Stride 2

In [4]:
# #Either
# basic_model = keras.models.load_model('checkpoints/ghostnetv1_w1.3_s2.weights.weights.h5', compile=False)

In [5]:
# #If the above did not work, then you need to build the model then load the weights. e.g.,
# basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
# basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
# basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model, target_activation='PReLU')

# basic_model.load_weights('checkpoints/ghostnetv1_w1.3_s2.h5')

Stride 1

In [6]:
#Either
# basic_model = keras.models.load_model('checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5', compile=False)
# checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5

In [7]:
import os
import sys
import numpy as np
import tensorflow as tf

weights_file = "./checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5"  # Adjust the path if necessary
# weights_file = "./checkpoints/ghostnetv1_w1.3_s1.weights_basic_model_latest.h5"  # Adjust the path if necessary
# Add the path to the 'backbones' folder to the system path
sys.path.append(os.path.abspath('./backbones'))

# Import the GhostNet architecture
from ghost_model import GhostNet

# Define paths


# Model parameters
input_shape = (224, 224, 3)  # Adjust this based on your input size
num_classes = 10           # Adjust this based on your dataset, set to 0 if not including top layer

# Initialize model as None
model = None

# Instantiate the GhostNet model
try:
    # Create the model directly with input_shape
    model = GhostNet(
        input_shape=input_shape,  # Input shape is used directly
        include_top=False,         # Set to False if you're using it as a feature extractor
        classes=num_classes,      # Number of output classes
        width=1.0,
        strides=1,                                 # Width multiplier as specified in the model
        name="GhostNet"         # Optional name for the model
    )
    print("GhostNetV1 model instantiated successfully!")
except Exception as e:
    print(f"Error instantiating GhostNetV1 model: {e}")

# Check if the model was successfully created
if model is not None:
    # Load pre-trained weights
    try:
        model.load_weights(weights_file)
        print("Weights loaded successfully from:", weights_file)
    except Exception as e:
        print(f"Error loading weights: {e}")

    # Compile the model (if necessary)
    try:
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        print("Model compiled successfully!")
    except Exception as e:
        print(f"Error compiling the model: {e}")

    # Check the model summary
    # print("Model Summary:")
    # model.summary()

GhostNetV1 model instantiated successfully!
Weights loaded successfully from: ./checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
Model compiled successfully!


In [8]:
# #If the above did not work, then you need to build the model then load the weights. e.g.,
# basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
# basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
# basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model, target_activation='PReLU')

# basic_model.load_weights('checkpoints/ghostnetv1_w1.3_s1.weights_basic_model_latest.h5')

In [9]:
# # import numpy as np
# # from tensorflow.keras.models import load_model


# # # Load the .bin file as embeddings
# # embeddings = np.fromfile('datasets/faces_emore/lfw.bin', dtype=np.float32)

# # # Print the total number of elements
# # print(f"Total number of elements: {embeddings.size}")

# # # Adjust the number of elements to be divisible by 128 (embedding dimension)
# # embedding_size = 128
# # num_elements = (embeddings.size // embedding_size) * embedding_size  # Round down to nearest multiple of 128

# # # Slice the array to ensure it's divisible by 128
# # embeddings = embeddings[:num_elements]

# # # Reshape the embeddings
# # N = embeddings.size // embedding_size  # Number of samples
# # embeddings = embeddings.reshape((N, embedding_size))

# # print(f"Reshaped embeddings to: {embeddings.shape}")


# # # Load your model (ensure it's trained and compatible with these embeddings)
# # # model = load_model('path_to_your_model.h5')

# # # Assuming you want to evaluate the model with these embeddings:
# # # Evaluate or pass the embeddings to the model for inference, e.g., using a callback
# # result = model.evaluate(embeddings)  # This is just an example; modify as per your use case
# # print(f"Evaluation result: {result}")

# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.ghostnet import preprocess_input
# import numpy as np

# # Load an image (you can iterate through all images in your dataset)
# img_path = './test_images/2_1.jpg'  # Replace with your image path
# img = image.load_img(img_path, target_size=(224, 224))  # Resize the image
# img_array = image.img_to_array(img)  # Convert the image to a NumPy array
# img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
# img_array = preprocess_input(img_array)  # Preprocess the image (depends on the model)

# # Use the model to predict
# result = model.predict(img_array)
# print(f"Prediction result: {result}")
import numpy as np

# Load the .bin file using numpy (assuming it is a serialized numpy array)
bin_file_path = 'datasets/faces_emore/lfw.bin'

# Load the binary data
data = np.fromfile(bin_file_path, dtype=np.float32)  # Adjust dtype as needed (check the file's structure)

# Print the shape of the loaded data
print("Shape of the loaded data:", data.shape)

# If the data is multi-dimensional, you can check its dimensions and count the elements
if len(data.shape) > 1:
    print(f"Total number of elements (pairs) in the .bin file: {data.shape[0]}")
else:
    # If the data is 1D, we simply count the number of elements
    print(f"Total number of elements in the .bin file: {len(data)}")





Shape of the loaded data: (10506,)
Total number of elements in the .bin file: 10506


In [10]:
ee = evals.eval_callback(model, 'datasets/faces_emore/lfw.bin', batch_size=128, flip=False, PCA_acc=True)
# df = 'datasets/faces_emore/lfw.bin'
# print(df)
# print(ee)
ee.on_epoch_end(0)

First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)


ValueError: invalid literal for int() with base 10: 'datasets/faces_emore_112x112_folders/00000/0_5.jpg'

In [ ]:
ee = evals.eval_callback(model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
# ee = evals.eval_callback(model, 'datasets/faces_emore/vgg2_fp.bin', batch_size=256, flip=True, PCA_acc=False)
# ee.on_epoch_end(0)

In [ ]:
# ee = evals.eval_callback(model, 'datasets/faces_emore/cfp_ff.bin', batch_size=256, flip=True, PCA_acc=False)
# ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(model, 'datasets/faces_emore/cfp_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/calfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cplfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/agedb_30.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='path_to_IJB_Dataset/ijb-testsuite/ijb', subset='IJBB', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBB"], label=tt.label)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='C:/Users/mohda/Downloads/ijb-testsuite/ijb', subset='IJBC', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBC"], label=tt.label)

Plot face quality distribution using norm value of feature

In [ ]:
cc = tf.norm(ee.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='agedb_30 quality')
cc = tf.norm(tt.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='IJBC quality')
plt.legend()
plt.tight_layout()